<a href="https://colab.research.google.com/github/mahima-c/DL-Problem-solution/blob/main/Many_to_One_%E2%80%93_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our dataset is the Sentiment labeled sentences dataset on the UCI Machine Learning Repository [20], a set of 3,000 sentences from reviews on Amazon, IMDb, and Yelp, each labeled with 0 if it expresses a negative sentiment, or 1 if it expresses a positive sentiment.

In [1]:
import numpy as np
import os
import shutil
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix

In [8]:
def clean_logs(data_dir):
    logs_dir = os.path.join(data_dir, "logs")
    shutil.rmtree(logs_dir, ignore_errors=True)
    return logs_dir


The dataset is provided as a zip file, which expands into a folder containing three files of labeled sentences, one for each provider, with one sentence and label per line, with the sentence and label separated by the tab character. We first download the zip file, then parse the files into a list of (sentence, label) pairs:

In [2]:
def download_and_read(url):
   local_file = url.split('/')[-1]
   local_file = local_file.replace("%20", " ")
   p = tf.keras.utils.get_file(local_file, url,
       extract=True, cache_dir=".")
   local_folder = os.path.join("datasets", local_file.split('.')[0])
   labeled_sentences = []
   for labeled_filename in os.listdir(local_folder):
       if labeled_filename.endswith("_labelled.txt"):
           with open(os.path.join(
                   local_folder, labeled_filename), "r") as f:
               for line in f:
                   sentence, label = line.strip().split('\t')
                   labeled_sentences.append((sentence, label))
   return labeled_sentences
labeled_sentences = download_and_read(      
    "https://archive.ics.uci.edu/ml/machine-learning-databases/" + 
    "00331/sentiment%20labelled%20sentences.zip")
sentences = [s for (s, l) in labeled_sentences]
labels = [int(l) for (s, l) in labeled_sentences]

90112/84188 [================================] - 0s 3us/step


Our objective is to train the model so that, given a sentence as input, it learns to predict the corresponding sentiment provided in the label. Each sentence is a sequence of words. However, in order to input it into the model, we have to convert it into a sequence of integers. Each integer in the sequence will point to a word. The mapping of integers to words for our corpus is called a vocabulary. Thus we need to tokenize the sentences and produce a vocabulary. This is done using the following code:

In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_counts)
print("vocabulary size: {:d}".format(vocab_size))
word2idx = tokenizer.word_index
idx2word = {v:k for (k, v) in word2idx.items()}

vocabulary size: 5271


Our vocabulary consists of 5271 unique words. It is possible to make the size smaller by dropping words that occur fewer than some threshold number of times, which can be found by inspecting the tokenizer.word_counts dictionary. In such cases, we need to add 1 to the vocabulary size for the UNK (unknown) entry, which will be used to replace every word that is not found in the vocabulary.

Each sentence can have a different number of words. Our model will require us to provide sequences of integers of identical length for each sentence. In order to support this requirement, it is common to choose a maximum sequence length that is large enough to accommodate most of the sentences in the training set. Any sentences that are shorter will be padded with zeros, and any sentences that are longer will be truncated. An easy way to choose a good value for the maximum sequence length is to look at the sentence length (in number of words) at different percentile positions:

In [4]:
seq_lengths = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(seq_lengths, p)) for p
   in [75, 80, 90, 95, 99, 100]])

[(75, 16.0), (80, 18.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


The preceding blocks of code can be run interactively multiple times to choose good values of vocabulary size and maximum sequence length respectively. In our example, we have chosen to keep all the words (so vocab_size = 5271), and we have set our max_seqlen to 64.

Our next step is to create a dataset that our model can consume. We first use our trained tokenizer to convert each sentence from a sequence of words (sentences) to a sequence of integers (sentences_as_ints), where each corresponding integer is the index of the word in the tokenizer.word_index. It is then truncated and padded with zeros. The labels are also converted to a NumPy array labels_as_ints, and finally, we combine the tensors sentences_as_ints and labels_as_ints to form a TensorFlow dataset:

In [5]:
max_seqlen = 64
# create dataset
sentences_as_ints = tokenizer.texts_to_sequences(sentences)
sentences_as_ints = tf.keras.preprocessing.sequence.pad_sequences(
   sentences_as_ints, maxlen=max_seqlen)
labels_as_ints = np.array(labels)
dataset = tf.data.Dataset.from_tensor_slices(
   (sentences_as_ints, labels_as_ints))

We want to set aside 1/3 of the dataset for evaluation. Of the remaining data, we will use 10% as an inline validation dataset that the model will use to gauge its own progress during training, and the remaining as the training dataset. Finally, we create batches of 64 sentences for each dataset:

In [6]:
dataset = dataset.shuffle(10000)
test_size = len(sentences) // 3
val_size = (len(sentences) - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)
batch_size = 64
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [9]:
class SentimentAnalysisModel(tf.keras.Model):
   def __init__(self, vocab_size, max_seqlen, **kwargs):
       super(SentimentAnalysisModel, self).__init__(**kwargs)
       self.embedding = tf.keras.layers.Embedding(
           vocab_size, max_seqlen)
       self.bilstm = tf.keras.layers.Bidirectional(
           tf.keras.layers.LSTM(max_seqlen)
       )
       self.dense = tf.keras.layers.Dense(64, activation="relu")
       self.out = tf.keras.layers.Dense(1, activation="sigmoid")
   def call(self, x):
       x = self.embedding(x)
       x = self.bilstm(x)
       x = self.dense(x)
       x = self.out(x)
       return x


In [11]:
# set random seed
tf.random.set_seed(42)
# clean up log area
data_dir = "./data"
logs_dir = clean_logs(data_dir)

In [12]:
# define model
# vocab_size + 1 to account for PAD character
model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
model.build(input_shape=(batch_size, max_seqlen))
model.summary()


Model: "sentiment_analysis_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  337408    
_________________________________________________________________
bidirectional_2 (Bidirection multiple                  66048     
_________________________________________________________________
dense_4 (Dense)              multiple                  8256      
_________________________________________________________________
dense_5 (Dense)              multiple                  65        
Total params: 411,777
Trainable params: 411,777
Non-trainable params: 0
_________________________________________________________________


In [13]:
# compile
model.compile(
    loss="binary_crossentropy",
    optimizer="adam", 
    metrics=["accuracy"]
)

In [14]:
# train
best_model_file = os.path.join(data_dir, "best_model.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file,
    save_weights_only=True,
    save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs, 
    validation_data=val_dataset,
    callbacks=[checkpoint, tensorboard])

Epoch 1/10
29/29 [==============================] - 8s 142ms/step - loss: 0.6924 - accuracy: 0.5163 - val_loss: 0.6868 - val_accuracy: 0.6650
Epoch 2/10
29/29 [==============================] - 3s 95ms/step - loss: 0.6729 - accuracy: 0.6772 - val_loss: 0.5025 - val_accuracy: 0.8650
Epoch 3/10
29/29 [==============================] - 3s 94ms/step - loss: 0.4496 - accuracy: 0.8371 - val_loss: 0.1766 - val_accuracy: 0.9350
Epoch 4/10
29/29 [==============================] - 3s 98ms/step - loss: 0.2238 - accuracy: 0.9280 - val_loss: 0.1308 - val_accuracy: 0.9600
Epoch 5/10
29/29 [==============================] - 3s 98ms/step - loss: 0.1453 - accuracy: 0.9560 - val_loss: 0.1341 - val_accuracy: 0.9550
Epoch 6/10
29/29 [==============================] - 3s 96ms/step - loss: 0.1139 - accuracy: 0.9657 - val_loss: 0.0591 - val_accuracy: 0.9850
Epoch 7/10
29/29 [==============================] - 3s 96ms/step - loss: 0.0797 - accuracy: 0.9762 - val_loss: 0.0637 - val_accuracy: 0.9750
Epoch 8/10
2

Our checkpoint callback has saved the best model based on the lowest value of validation loss, and we can now reload this for evaluation against our held out test set:

In [15]:
# evaluate with test set
best_model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
best_model.build(input_shape=(batch_size, max_seqlen))
best_model.load_weights(best_model_file)
best_model.compile(
    loss="binary_crossentropy",
    optimizer="adam", 
    metrics=["accuracy"]
)

test_loss, test_acc = best_model.evaluate(test_dataset)
print("test loss: {:.3f}, test accuracy: {:.3f}".format(test_loss, test_acc))

16/16 [==============================] - 2s 24ms/step - loss: 0.0326 - accuracy: 0.9885
test loss: 0.034, test accuracy: 0.993


In [17]:
# predict on batches
labels, predictions = [], []
idx2word[0] = "PAD"
is_first_batch = True
for test_batch in test_dataset:
    inputs_b, labels_b = test_batch
    pred_batch = best_model.predict(inputs_b)
    predictions.extend([(1 if p > 0.5 else 0) for p in pred_batch])
    labels.extend([l for l in labels_b])
    if is_first_batch:
        for rid in range(inputs_b.shape[0]):
            words = [idx2word[idx] for idx in inputs_b[rid].numpy()]
            words = [w for w in words if w != "PAD"]
            sentence = " ".join(words)
            print("{:d}\t{:d}\t{:s}".format(labels[rid], predictions[rid], sentence))
        is_first_batch = False

print("accuracy score: {:.3f}".format(accuracy_score(labels, predictions)))
print("confusion matrix")
print(confusion_matrix(labels, predictions))


0	0	so don't go there if you are looking for good food
1	1	portable and it works
0	0	the pan cakes everyone are raving about taste like a sugary disaster tailored to the palate of a six year old
0	0	will not be back
1	1	great it was new packaged nice works good no problems and it came in less time then i expected
1	1	this is the phone to get for 2005 i just bought my s710a and all i can say is wow
0	0	the lines the cuts the audio everything is wrong
1	1	this if the first movie i've given a 10 to in years
0	0	all in all ha long bay was a bit of a flop
0	0	speaking of the music it is unbearably predictably and kitchy
0	0	amazon sucks
1	1	great charger
0	0	the commercials are the most misleading
1	1	very clear quality sound and you don't have to mess with the sound on your ipod since you have the sound buttons on the headset
1	1	i ordered the voodoo pasta and it was the first time i'd had really excellent pasta since going gluten free several years ago
1	1	it is wonderful and inspiring to